In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

# Read in data
z_data = pd.read_csv('fse17_act_deact_zscore_groups.csv',sep=';',decimal='.')

# Remove irrelevant columns
z_data.drop(list(z_data.filter(regex = '_act')),axis = 1, inplace = True)

#first, drop everything non-numeric ===========================================================
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('response', axis = 1)
z_data = z_data.drop('snippet', axis = 1)

numLabels = 2 # num of different labels (comprehension and rest)
sizeTrainSet = 11 # num of participants used for training
sizeTestSet = 3 # num participants used for testing
numParticipants = sizeTrainSet + sizeTestSet

C:\Users\author\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\author\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnin

In [2]:
#group the data and get the labels after the grouping
#return the grouped data and the labels
def prepare(data,groupingColumns):
    grouped = data.groupby(groupingColumns)
    groupedAgg = grouped.aggregate(np.mean)
    labels = groupedAgg.index.get_level_values(level='task')
    return groupedAgg,labels

In [3]:
#split according to participants (first 12 and last 4)
#return all four sets (much like train_test_split from sklearn)
#TODO: introduce variation (see file coarseAverageParticipantSplit)

def split(features, labels, low, high):
    
    #=== creating training and validation set ======================================================
    # Saving feature names for later use
    feature_list = list(features.columns)

    # Convert to numpy array
    features = np.array(features)
    labels = np.array(labels)

    #get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
    training_features = features[0:low]
    training_features = np.array(training_features)

    testing_features = features[low:high]
    testing_features = np.array(testing_features)

    training_target = labels[0:low]
    training_target = np.array(training_target)
    testing_target = labels[low:high]
    testing_target = np.array(testing_target)
    #=== end creating training and validation set ===================================================
    
    return training_features, testing_features, training_target, testing_target

In [4]:
def findIndices(name):
    numGroups = int(name.split('_')[1])
    low = sizeTrainSet*numGroups*numLabels
    high = numParticipants*numGroups*numLabels
    return low, high



In [ ]:
import pdb

#define the columns to group by
groupingColumn1 = 'proband'
groupingColumn2 = 'task'
groupingColumn3 = ''

#define how many different variants of aggregation and aggregation levels
numVariationsPerAggLevel = 10
numAggLevels = 40
numAggregationVariants = numVariationsPerAggLevel * numAggLevels

#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')
print(offset)
collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):
    # group the data set
#     pdb.set_trace()
    #=============================================================================================================
    columnsForGrouping = [groupingColumn1,groupingColumn2,z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================
    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    print(offset)
    print(low, high)
    
    folder = 'Deact/'
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'
    #=============================================================================================================
    # run t_pot
    # TODO: extract method
    tpot = TPOTClassifier(generations=5, population_size=5, verbosity=2, n_jobs=20,max_eval_time_mins=1)
    tpot.fit(training_features, training_target)
    print(tpot.score(testing_features, testing_target))
    tpot.export(folder + fileName)
#     #=============================================================================================================
    
    offset = offset + 1


1800
1800
44 56


C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: 0.9800000000000001


Generation 2 - Current best internal CV score: 0.9800000000000001


Generation 3 - Current best internal CV score: 0.9800000000000001


...

Generation 1 - Current best internal CV score: 0.86


Generation 2 - Current best internal CV score: 0.8797368421052632


Generation 3 - Current best internal CV score: 0.8850000000000001


Generation 4 - Current best internal CV score: 0.8850000000000001


Optimization Progress:  83%|████████████████████████████████████████████▏        | 25/30 [01:09<00:20,  4.03s/pipeline]

In [5]:
#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')
print(offset)
collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):    

    #=============================================================================================================
    columnsForGrouping = ['proband','task',z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================
    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'

    #=============================================================================================================
    #do the learning
    # TODO: extract method
    with open(folder + fileName) as f:
        content = f.readlines()
    cleanedContent = []
    for line in content:
        if 'tpot_data' not in line and 'training_target, testing_target' not in line:
            cleanedContent.append(line)
    
    fileNameCleaned = folder + 'cleaned_' + fileName
    with open(fileNameCleaned, 'w') as filehandle:  
        for line in cleanedContent:
            filehandle.write('%s\n' % line)
            
    %run -i $fileNameCleaned
    
    # print the accuracy
    # TODO: extract method
    # TODO: print all accurracy values to one file
    num_matches = 0;
    for a, b in zip(testing_target, results):
        if a == b:
            num_matches = num_matches + 1
        else:
            unmatched.append((a,b))
    print('Number of matches:',num_matches,'(of',testing_target.size,')')

    accuracy = num_matches/testing_target.size*100
    print('Accuary: ',accuracy)
    #=============================================================================================================
    
    #=============================================================================================================
    #collect all accuracy values to plot them
    collectAccuracies.append(accuracy)
    
    
    offset = offset + 1
    

1800
1800
44 56


C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Number of matches: 12 (of 12 )
Accuary:  100.0
1801
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1802
44 56


C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Number of matches: 12 (of 12 )
Accuary:  100.0
1803
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1804
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1805
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1806
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1807
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1808
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1809
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
1810
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
1811
66 84
Number of matches: 16 (of 18 )
Accuary:  88.88888888888889
1812
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
1813
66 84
Number of matches: 17 (of 18 )
Accuary:  94.44444444444444
1814
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
1815
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
1816
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
1817
66 84
Number of matches: 17 (of 18 )
Accuary:  94.44444444444444
1818
66 84
Number of matches: 17 (of 18 )
Accua

...

Number of matches: 46 (of 54 )
Accuary:  85.18518518518519
1821
198 252
Number of matches: 46 (of 54 )
Accuary:  85.18518518518519
1822
198 252
Number of matches: 48 (of 54 )
Accuary:  88.88888888888889
1823
198 252
Number of matches: 48 (of 54 )
Accuary:  88.88888888888889
1824
198 252


C:\Users\author\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 50 (of 54 )
Accuary:  92.5925925925926
1825
198 252
Number of matches: 49 (of 54 )
Accuary:  90.74074074074075
1826
198 252
Number of matches: 41 (of 54 )
Accuary:  75.92592592592592
1827
198 252


C:\Users\author\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 50 (of 54 )
Accuary:  92.5925925925926
1828
198 252
Number of matches: 46 (of 54 )
Accuary:  85.18518518518519
1829
198 252
Number of matches: 49 (of 54 )
Accuary:  90.74074074074075
1830
286 364
Number of matches: 70 (of 78 )
Accuary:  89.74358974358975
1831
286 364
Number of matches: 66 (of 78 )
Accuary:  84.61538461538461
1832
286 364
Number of matches: 67 (of 78 )
Accuary:  85.8974358974359
1833
286 364
Number of matches: 60 (of 78 )
Accuary:  76.92307692307693
1834
286 364
Number of matches: 59 (of 78 )
Accuary:  75.64102564102564
1835
286 364
Number of matches: 66 (of 78 )
Accuary:  84.61538461538461
1836
286 364
Number of matches: 66 (of 78 )
Accuary:  84.61538461538461
1837
286 364
Number of matches: 63 (of 78 )
Accuary:  80.76923076923077
1838
286 364
Number of matches: 69 (of 78 )
Accuary:  88.46153846153845
1839
286 364
Number of matches: 63 (of 78 )
Accuary:  80.76923076923077


In [6]:
len(unmatched)

203

In [7]:
countComprehensionRest = 0
countComprehensionSyntax = 0
countRestSyntax = 0
for x in unmatched:
    if x == (0,1) or x == (1,0):
        countComprehensionRest = countComprehensionRest + 1
    if x == (0,2) or x == (2,0):
        countComprehensionSyntax = countComprehensionSyntax + 1
    if x == (1,2) or x == (2,1):
        countRestSyntax = countRestSyntax + 1
print('Comprehension/Rest',countComprehensionRest)
print('Comprehension/Syntax',countComprehensionSyntax)
print('Rest/Syntax',countRestSyntax)

Comprehension/Rest 203
Comprehension/Syntax 0
Rest/Syntax 0


In [8]:
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

In [9]:
list1, list2, list3, list4 = split_list(collectAccuracies,4)
frames = [list1, list2, list3, list4]
collectedAccuracies = pd.DataFrame(frames).T
collectedAccuracies.columns = ['2_groups_Deact','3_groups_Deact','9_groups_Deact','13_groups_Deact']


collectedAccuracies.to_csv('triangulate_Deact.csv',sep=';',decimal='.', index=False)